# Spark Files Read/Write ClassTester

Test the DataIO controller that serves as wrapper to read and write data from and to a particular file system.

### Dependencies

* Spark Hadoop 3.0 or latest
* [AWS jars files](https://stackoverflow.com/questions/44411493/java-lang-noclassdeffounderror-org-apache-hadoop-fs-storagestatistics) must be in $SPARK_HOME/jars  folder. The jar versions must match (e.g. 3.3.4 or 1.12.376 as below)
   * hadoop-aws-3.3.4.jar
   * hadoop-common-3.3.4.jar
   * aws-java-sdk-s3-1.12.376.jar
   * aws-java-sdk-dynamodb-1.12.376.jar
   * aws-java-sdk-core-1.12.376.jar
   * jets3t-0.7.1.jar
   * 

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True


In [2]:
import os
import sys

proj_dir = os.path.abspath(os.pardir)
sys.path.insert(1,proj_dir.split('rezaware/')[0])
from rezaware.modules.etl.loader import sparkFILEwls as spark
from rezaware.modules.etl.loader.sparkFILEwls import credentials as cred

''' restart initiate classes '''
if debug:
    import importlib
    spark= importlib.reload(spark)

__desc__ = "read and write files from and to a particular source"
clsSpark = spark.FileWorkLoads(desc=__desc__)
if clsSpark._session:
    clsSpark._session.stop
print("\n%s class initialization and load complete!" % __desc__)

All functional SPARKFILEWLS-libraries in LOADER-package of ETL-module imported successfully!
All functional SPARKFILEWLS-libraries in LOADER-package of ETL-module imported successfully!
All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
sparkFILEwls Class initialization complete

read and write files from and to a particular source class initialization and load complete!


In [3]:
# clsSpark.storeMode = "local-fs"
clsSpark.storeMode = "aws-s3-bucket"
# clsSpark.storeMode = "google-storage"
print("mode =",clsSpark.storeMode)

''' set the driver '''
if clsSpark.storeMode.lower() == "local-fs":
    clsSpark.jarDir = os.path.join(os.environ['SPARK_HOME'],"jars",
                                   "postgresql-42.6.0.jar")
elif clsSpark.storeMode.lower() == "aws-s3-bucket":
    clsSpark.jarDir = os.path.join(os.environ['SPARK_HOME'],"jars",
                                   "aws-java-sdk-s3-1.12.376.jar")
elif clsSpark.storeMode.lower() == "google-storage":
    clsSpark.jarDir = os.path.join(os.environ['SPARK_HOME'],"jars",
                                   "gcs-connector-hadoop3-2.2.10.jar")
else:
    pass
print("jar =",clsSpark.jarDir)

mode = aws-s3-bucket
jar = /opt/spark_hadoop_3/jars/aws-java-sdk-s3-1.12.376.jar


In [4]:
### local-fs
# clsSpark.storeRoot = proj_dir.split('rezaware/')[0]
### AWS S3
clsSpark.storeRoot = "rezgcorp-data-science"
### google cloud store
# clsSpark.storeRoot = "tip-daily-marketcap"   #"rezaware-wrangler-source-code"
print("data_root =",clsSpark.storeRoot)

data_root = rezgcorp-data-science


## Read from storage and return as dtype

In [5]:
__as_type__ = "pandas"  # specify data type to return from read file
''' set the folder path relative the storeRoot '''
_folder_path = "Booking Data/2023-02-23_12/"
''' set the single file name '''
_file_name = None  #file name to read from or write data to; e.g. myfile.csv
''' set the file type to read all files of specified type in folder '''
_file_type = "csv"   # specify file type to write data to or read from
''' options key value pairs must match the pyspark standard'''
options = {
    "inferSchema":True,
    "header":True,
    "delimiter":",",
    "pathGlobFilter":'*.csv',
    "recursiveFileLookup":True,
}

_data = clsSpark.read_files_to_dtype(
    as_type=__as_type__,
    folder_path=_folder_path,
    file_name=_file_name,
    file_type=_file_type,
    **options,
)
if __as_type__ == 'pandas':
    print("got back {0} rows with return dtype {1}"
          .format(_data.shape[0],__as_type__))
    print(_data.head(5))
elif __as_type__ == 'spark':
    print("got back {0} rows with return dtype {1}"
          .format(_data.count(),__as_type__))
    print(_data.show(5,truncate=False))
elif __as_type__ == 'dict':
    print("got back {0} rows with return dtype {1}"
          .format(len(_data),__as_type__))
    print(_data[:3])
else:
    print('Unrecognised __as_type__' % __as_type__)

23/07/06 21:05:10 WARN Utils: Your hostname, FarmRaider2 resolves to a loopback address: 127.0.1.1; using 192.168.124.14 instead (on interface enp3s0)
23/07/06 21:05:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/07/06 21:05:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


sparkFILEwls Class initialization complete


23/07/06 21:05:14 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


got back 61747 rows with return dtype pandas
           Search Date Check-in Date Check-out Date  \
0  02-23-2023 12:00:09    2023-02-23     2023-02-24   
1  02-23-2023 12:00:09    2023-02-23     2023-02-24   
2  02-23-2023 12:00:09    2023-02-23     2023-02-24   
3  02-23-2023 12:00:09    2023-02-23     2023-02-24   
4  02-23-2023 12:00:09    2023-02-23     2023-02-24   

                                               Hotel Star Rating(out of 5)  \
0                              Caribe Royale Orlando                     4   
1                       Garnet Inn & Suites, Orlando                  None   
2  Red Roof Inn PLUS+ Orlando - Convention Center...                     3   
3                      Quality Inn & Suites Downtown                     2   
4  Travelodge by Wyndham Orlando at Heart of Inte...                     2   

                                           Room Type    Rate  Review Score  \
0               Newly Re-Imagined Caribe Queen Suite  US$302           8.8   

## Filter and save data in storage location

In [13]:
# import pandas as pd

_filter_data = _data.loc[_data['asset_symbol'].isin(['1inch','zks'])]
__file_name__="savetodummy.csv"
print("writing %d filtered rows to file %s in %s" 
      % (_filter_data.shape[0],__file_name__,__local_folder_path__))
write_data=clsSpark.write_data(
    file_name=__file_name__,
    folder_path=__local_folder_path__,
    data=_filter_data
)

print("saved to %s " % write_data)

writing 60 filtered rows to file savetodummy.csv in rezaware/data/etl/loader/sampledata/
saved to <class 'str'> 


In [135]:
from google.cloud import storage


def write_read(bucket_name, blob_name):
    """Write and read a blob from GCS using file-like IO"""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The ID of your new GCS object
    # blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    # Mode can be specified as wb/rb for bytes mode.
    # See: https://docs.python.org/3/library/io.html
#     with blob.open("w") as f:
#         f.write("Hello world")

    with blob.open("r") as f:
        print(f.read())
